# Naive Bayes (the easy way)

In [102]:
import os
import io
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)
            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []}) 
data = pd.concat([data, dataFrameFromDirectory("emails/spam", "spam")]);
data = pd.concat([data, dataFrameFromDirectory("emails/ham", "ham")])

In [103]:
data['spam']=data['class'].apply(lambda x: 1 if x=='spam' else 0)
data

,message,class,spam
emails/spam/00249.5f45607c1bffe89f60ba1ec9f878039a,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",spam,1
emails/spam/00373.ebe8670ac56b04125c25100a36ab0510,ATTENTION: This is a MUST for ALL Computer Use...,spam,1
emails/spam/00214.1367039e50dc6b7adb0f2aa8aba83216,This is a multi-part message in MIME format.\n...,spam,1
emails/spam/00210.050ffd105bd4e006771ee63cabc59978,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,spam,1
emails/spam/00033.9babb58d9298daa2963d4f514193d7d6,This is the bottom line. If you can GIVE AWAY...,spam,1
...,...,...,...
emails/ham/01037.6b42b5f3d3d9e6293bf24af66b250655,"On Thu, 2002-08-29 at 00:02, Ville Skyttä wrot...",ham,0
emails/ham/02056.7bc7703e40a24dda665d4ce7b0cba710,"URL: http://www.newsisfree.com/click/-6,835599...",ham,0
emails/ham/01782.278f53b8f65fcd422cb26c5bbe74599d,use Perl Daily Newsletter\n\n\n\nIn this issue...,ham,0
emails/ham/00043.d2673a72d215cbdd747dc98cde41fbd2,< >\n\n> I downloaded a driver from the nVidia...,ham,0


In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data.message,data.spam, test_size=0.2)

In [105]:
vectorizer = CountVectorizer()
message_counts = vectorizer.fit_transform(X_train)
print(message_counts)

  (0, 52212)	1
  (0, 26953)	1
  (0, 55471)	1
  (0, 36500)	1
  (0, 15736)	1
  (0, 15350)	1
  (0, 6175)	1
  (0, 1245)	1
  (0, 17515)	1
  (0, 36969)	1
  (0, 48473)	1
  (0, 36460)	1
  (0, 9979)	1
  (0, 37735)	2
  (0, 45488)	1
  (0, 28236)	1
  (0, 46513)	1
  (0, 43564)	1
  (0, 38021)	1
  (0, 10550)	1
  (0, 32024)	1
  (0, 8704)	1
  (0, 49707)	1
  (0, 47350)	1
  (0, 45351)	1
  :	:
  (2399, 44675)	1
  (2399, 11809)	1
  (2399, 43853)	2
  (2399, 17626)	1
  (2399, 41871)	1
  (2399, 24533)	1
  (2399, 12878)	1
  (2399, 47421)	1
  (2399, 183)	1
  (2399, 21247)	2
  (2399, 15428)	1
  (2399, 21591)	1
  (2399, 4072)	1
  (2399, 43919)	1
  (2399, 44398)	1
  (2399, 53178)	3
  (2399, 52864)	1
  (2399, 40897)	1
  (2399, 3998)	1
  (2399, 1228)	4
  (2399, 3999)	1
  (2399, 4000)	1
  (2399, 38302)	1
  (2399, 20925)	2
  (2399, 4001)	1


In [106]:
classifier = MultinomialNB()
target_class = Y_train

In [107]:
classifier.fit(message_counts, target_class)

MultinomialNB()

In [108]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts1 = vectorizer.transform(examples)
predictions = classifier.predict(example_counts1)
predictions

array([1, 0])

In [109]:
example_counts = vectorizer.transform(X_test)
predictions = classifier.predict(example_counts)
predictions

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [110]:
classifier.score(example_counts,Y_test)

0.9616666666666667